In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re

import liikutils as lu

In [2]:
lu.load_data()

In [3]:
pd.Series(lu.nykytila_df.columns)

0                                              Koulu ID
1                            Viimeisin (1= kyllä, 0=ei)
2                 Koulumuoto (1=ala, 2=ylä, 3=yhtenäis)
3             oppilaslkm (1= <100, 2= 100-300, 3= >300)
4                                    Vastaamisajankohta
5     Koulumuoto, jota arviointi koskee (1=ala, 2=yl...
6     1.1 Koululla on tiimi, joka koordinoi Liikkuva...
7     1.2 Liikkumisen edistämiseksi on valittu lukuv...
8     1.3 Koulupäivän aikainen liikkuminen on kirjat...
9     1.4 Hyvinvoinnin ja liikkumisen edistäminen on...
10    1.5 Liikkuva koulu -toiminta on kirjattu kunta...
11    1.6 Rehtori on sitoutunut Liikkuva koulu -toim...
12    1.7 Opettajia osallistuu liikunnallisiin tai t...
13    2.1 Opetuksessa on sovittu käytännöistä, joill...
14    2.2 Toiminnallisia menetelmiä hyödynnetään eri...
15    2.3 Teknologiaa hyödynnetään liikkeen lisääjän...
16    2.4 Koulun opetustiloja on muokattu siten, ett...
17    2.5 Koulun piha-aluetta ja lähiympäristöä 

In [4]:
np.array(lu.nykytila_df)[:,6:-1]

array([[4, 3, 4, ..., 2, 2, 2],
       [3, 2, 1, ..., 3, 3, 2],
       [1, 2, 1, ..., 1, 3, 4],
       ...,
       [4, 4, 4, ..., 1, 3, 1],
       [0, 0, 0, ..., 0, 1, 1],
       [2, 4, 3, ..., 0, 0, 1]], dtype=object)

In [5]:
# Esim ICA-analyysi samankaltaisten faktoreiden löytämiseksi

In [6]:
import sklearn as sk
from sklearn.decomposition import FastICA
ica = FastICA(n_components=3)


In [7]:
X = np.array(lu.nykytila_df)[:,6:13]
Y = ica.fit_transform(X)
df=pd.DataFrame(np.round(ica.components_,2))
df.columns=lu.nykytila_df.columns[6:13]
df.T

,0,1,2
"1.1 Koululla on tiimi, joka koordinoi Liikkuva koulu -toimintaa.",-0.22,0.52,0.18
1.2 Liikkumisen edistämiseksi on valittu lukuvuoden tavoitteet ja toimenpiteet.,-0.26,-0.03,0.14
1.3 Koulupäivän aikainen liikkuminen on kirjattu osaksi koulun lukuvuoden toimintasuunnitelmaa.,-0.29,-0.49,0.15
1.4 Hyvinvoinnin ja liikkumisen edistäminen on kirjattu koulun OPS:iin.,0.07,-0.49,0.19
"1.5 Liikkuva koulu -toiminta on kirjattu kuntastrategiaan, kunnan hyvinvointistrategiaan tai muuhun vastaavaan asiakirjaan.",0.71,0.03,0.33
1.6 Rehtori on sitoutunut Liikkuva koulu -toimintaan.,-0.05,0.11,0.15
1.7 Opettajia osallistuu liikunnallisiin tai toiminnallista oppimista tukeviin täydennyskoulutuksiin.,0.21,0.45,0.18


In [8]:
X = np.array(lu.nykytila_df)[:,13:18]
Y = ica.fit_transform(X)
df=pd.DataFrame(np.round(ica.components_,2))
df.columns=lu.nykytila_df.columns[13:18]
df.T

,0,1,2
"2.1 Opetuksessa on sovittu käytännöistä, joilla istumisjaksot oppitunneilla katkaistaan.",-0.43,-0.63,-0.16
2.2 Toiminnallisia menetelmiä hyödynnetään eri aineiden oppitunneilla.,-0.25,-0.15,0.06
2.3 Teknologiaa hyödynnetään liikkeen lisääjänä eri oppiaineissa.,-0.67,0.80,-0.22
"2.4 Koulun opetustiloja on muokattu siten, että ne lisäävät toiminnallisuutta oppitunneilla.",0.15,0.28,0.94
2.5 Koulun piha-aluetta ja lähiympäristöä hyödynnetään opetuksen tukena.,0.02,-0.62,0.15


In [9]:
uudet_nyky_df = lu.nykytila_df[lu.nykytila_df.Vastaamisajankohta.isin(['2023 kevät','2022 syksy'])]

In [10]:
np.array(uudet_nyky_df)[:,6:-1].sum(axis=1)

array([91, 103, 89, 79, 87, 111, 93, 79, 77, 115, 89, 89, 88, 98, 102, 65,
       115, 101, 102, 67, 101, 62, 101, 84, 85, 85, 84, 74, 100, 55, 56,
       99, 81, 96, 120, 105, 70, 112, 98, 109, 87, 97, 82, 10, 99, 116,
       106, 95, 112, 90, 88, 93, 93, 52, 81, 83, 91, 77, 100, 93, 96, 111,
       86, 68, 112, 89, 90, 79, 96, 93, 104, 101, 103, 77, 83],
      dtype=object)

In [11]:
nykysum_df = pd.DataFrame(zip(uudet_nyky_df['Koulu ID'],np.array(uudet_nyky_df)[:,6:-1].sum(axis=1)),columns=['Koulu_ID','nykytila_sum'])
nykysum_df.to_csv("../data/nykytila_summa.csv")

In [12]:
uudet_nyky_df.columns

Index(['Koulu ID', 'Viimeisin (1= kyllä, 0=ei)',
       'Koulumuoto (1=ala, 2=ylä, 3=yhtenäis)',
       'oppilaslkm (1= <100, 2= 100-300, 3= >300)', 'Vastaamisajankohta',
       'Koulumuoto, jota arviointi koskee (1=ala, 2=ylä, 3=yhtenäis 1-9, 4= yhtenäis 1-6, 5=yhtenäis 7-9)',
       '1.1 Koululla on tiimi, joka koordinoi Liikkuva koulu -toimintaa.',
       '1.2 Liikkumisen edistämiseksi on valittu lukuvuoden tavoitteet ja toimenpiteet.',
       '1.3 Koulupäivän aikainen liikkuminen on kirjattu osaksi koulun lukuvuoden toimintasuunnitelmaa.',
       '1.4 Hyvinvoinnin ja liikkumisen edistäminen on kirjattu koulun OPS:iin.',
       '1.5 Liikkuva koulu -toiminta on kirjattu kuntastrategiaan, kunnan hyvinvointistrategiaan tai muuhun vastaavaan asiakirjaan.',
       '1.6 Rehtori on sitoutunut Liikkuva koulu -toimintaan.',
       '1.7 Opettajia osallistuu liikunnallisiin tai toiminnallista oppimista tukeviin täydennyskoulutuksiin.',
       '2.1 Opetuksessa on sovittu käytännöistä, joilla is

In [13]:
nykytila_indeksit = {'strategia':'1.2  1.3  1.5  1.4  1.6', 
                    'olosuhteet':'2.4  2.5  4.4  4.5  4.6  4.7  5.2',
                    'toiminta':  '2.1  2.2  2.3  3.1  3.2  3.3  3.4  3.5  3.6  3.8  4.1  4.2  4.3  5.1  5.3  5.4  6.1',
                    'tuki':      '1.1  1.7  3.7  6.2  6.3  6.4  6.5' }




In [14]:
dfs = []
for indeksi in nykytila_indeksit.keys():
    summaindeksi = pd.DataFrame(uudet_nyky_df, columns = [c for c in uudet_nyky_df.columns if c[0:3] in nykytila_indeksit[indeksi].split(' ')])
    dfs.append(pd.DataFrame(summaindeksi.sum(axis=1),columns=[indeksi]))

indeksit_df = pd.concat(dfs,axis=1)
indeksit_df.index = uudet_nyky_df['Koulu ID']
indeksit_df.index.name='Koulu_ID'
indeksit_df.to_csv("../data/nykytila_indeksit.csv")
indeksit_df

,strategia,olosuhteet,toiminta,tuki
Koulu_ID,,,,
101,18,19,37,17
112,12,22,51,18
113,14,17,44,14
114,12,15,36,16
116,11,19,46,11
...,...,...,...,...
181,18,15,51,20
185,14,21,52,14
187,15,21,50,17


In [15]:
lu.nykytila_df[lu.nykytila_df['Koulu ID']==194]

,Koulu ID,"Viimeisin (1= kyllä, 0=ei)","Koulumuoto (1=ala, 2=ylä, 3=yhtenäis)","oppilaslkm (1= <100, 2= 100-300, 3= >300)",Vastaamisajankohta,"Koulumuoto, jota arviointi koskee (1=ala, 2=ylä, 3=yhtenäis 1-9, 4= yhtenäis 1-6, 5=yhtenäis 7-9)","1.1 Koululla on tiimi, joka koordinoi Liikkuva koulu -toimintaa.",1.2 Liikkumisen edistämiseksi on valittu lukuvuoden tavoitteet ja toimenpiteet.,1.3 Koulupäivän aikainen liikkuminen on kirjattu osaksi koulun lukuvuoden toimintasuunnitelmaa.,1.4 Hyvinvoinnin ja liikkumisen edistäminen on kirjattu koulun OPS:iin.,...,5.1 Oppilaita kannustetaan liikkumaan koulumatkat kävellen tai pyöräillen.,5.2 Koululla on toimivat polkupyörien ja kypärien säilytysratkaisut.,5.3 Koululla on liikunnallista ja muuta aktiivista kerhotoimintaa koulupäivän yhteydessä.,5.4 Koululla on kerhoja vähemmän liikunnallisille tai liikuntaa harrastamattomille oppilaille.,6.1 Opettajat tekevät keskenään yhteistyötä yli oppiainerajojen aktiivisuuden lisäämiseksi.,6.2 Huoltajien kanssa tehdään yhteistyötä kävelyn tai pyöräilyn edistämiseksi koulumatkoilla.,6.3 Kouluterveydenhoitaja on mukana oppilaiden liikkumisen edistämisessä.,"6.4 Koulu tekee yhteistyötä kunnan eri hallintokuntien kanssa (opetus-, nuoriso-, liikunta- sekä sosiaali- ja terveystoimi) liikunnallisen elämäntavan edistämiseksi.","6.5 Koulu tekee yhteistyötä paikallisten toimijoiden kanssa (mm. järjestöt, yritykset).",Vapaa sana
77,194,1,3,3.0,2021 Syksy,3,4,2,2,3,...,2,1,1,0,2,0,0,1,1,NaN
114,194,0,3,2.0,2020 syksy,3,4,4,2,4,...,2,0,0,0,1,1,1,2,1,NaN
139,194,1,3,2.0,2019 syksy,3,2,1,2,2,...,1,0,3,0,2,1,0,1,1,NaN
140,194,1,3,1.0,2019 syksy,3,2,3,3,3,...,4,4,4,4,4,4,1,4,3,NaN
141,194,1,3,2.0,2019 syksy,3,0,3,4,3,...,2,1,4,2,2,1,0,0,3,NaN
142,194,0,2,2.0,2019 syksy,2,4,2,4,3,...,3,4,1,1,2,1,3,1,2,NaN
